<img align="center" src="img/course.png" width="800">

# 16720 (B)  Object Tracking in Videos - Assignment 6 - Q5
    Instructor: Kris                          TAs: Arka, Rohan, Rawal, Sheng-Yu, Jinkun

In [ ]:
# Libraries

import json
import numpy as np
%matplotlib inline
from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.optimize import linear_sum_assignment
from scipy.interpolate import RectBivariateSpline

## Q5: Multi-Object Tracking by Detection (EC, 45 PT)

### Overview

In this extra credit problem, you will be introduced to a more modern perspective on tracking. In the previous problems, you implemented single-object tracking with a classical method, the LK tracker. Multi-object tracking (MOT), on the other hand, is a richer and more useful problem to attack. One approach to this problem is called tracking by detection. In this paradigm, for each frame of a video, we produce object detections (typically from a learned object detection neural net). These are called proposals, and are often bounding boxes. In the next step, we associate those boxes with any existing tracked objects. For a more in-depth overview, please see the following [link](https://arshren.medium.com/an-introduction-to-object-tracking-9fd6249a76b6).

### Q5.1 Loading the bounding boxes, video and visualization (5 pts)

In the spirit of the World Cup, we will be evaluating your method on a short excerpt from a [famous soccer clip](https://youtu.be/uBa8dYlqv8Y). We'll begin by loading and visualizing the input. We have already computed bounding boxes for you, which are available in ```soccer_boxes.json```. The images to use in the video are available in ```soccer_images.npy``` Both files can be download at this [link](https://www.dropbox.com/sh/uovrr3cgtehhtuc/AAATS-GtGEwfS-z2MXRNku7Ea?dl=0).

Fill in the functions below. For testing, your result for the visualization on the 124th frame should look something like

<img align="center" src="img/sample_bbox_img.jpg" width="800">

Please submit an image of the bounding boxes rendered on the 1st frame of the sequence along with your code for this question to the writeup PDF.

In [ ]:
def load_images_and_boxes(img_path, box_path):

    data = 'soccer_images'
    imgs = np.load(img_path % data,allow_pickle = True)

    data_box = 'soccer_boxes'
    f = open(box_path % data_box)
    boxes = json.load(f)
    
    return imgs, boxes
c = ['red','blue','grey','green','orange','black','white','brown','cyan','violet','yellow','purple']
def render_single_frame(image, bboxes, colors = False):

    if colors == 1:
        fig = plt.figure(1)
        ax = fig.add_subplot(111)
        for i in bboxes:
            ax.add_patch(patches.Rectangle((i[0],i[1]),i[2]-i[0]+1,i[3]-i[1]+1, 
                                           linewidth = 2, edgecolor = 'green', fill = False))
        plt.imshow(image)
        plt.show()
        ax.clear()
    else:
        fig = plt.figure(1)
        ax = fig.add_subplot(111)
        o = 0
        for i in bboxes:
            ax.add_patch(patches.Rectangle((i[0],i[1]),i[2]-i[0]+1,i[3]-i[1]+1,
                                           linewidth=2, edgecolor=colors[o], fill=False))
            
            o = o + 1
        plt.imshow(image)
        plt.show()
        ax.clear()

In [ ]:
a, b = load_images_and_boxes('./%s.npy','./%s.json')

In [ ]:
render_single_frame(a[124],b[124]['boxes'], colors = True)
render_single_frame(a[0],b[0]['boxes'], colors = True)

### Q5.2 Implementing a similarity metric (10 pts)

In order to do track association, we need a way to measure how similar two bounding boxes are to each other. One way to do this is intersection-over-union (IoU). An overview of how to compute IoU is provided [here](https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/). Below, you will implement a function to compute IoU. The input will be a set of N bounding boxes (representing the boxes in the reference frame), and a set of M bounding boxes (representing the boxes in the next frame) and the output will be an NxM matrix, with the ```[i, j]```th entry correspoding to bounding box ```i``` in the reference frame's IoU with bounding box ```j``` of the next frame.

For this question, please submit the matrix of IoUs between the boxes on the 124th and 125th frames, as well as your code to the writeup PDF, 

In [ ]:
def overlappingArea(l_1, r_1, l_2, r_2):
    xx = 0
    yy = 1

    area_ = abs(l_1[xx] - r_1[xx]) * abs(l_1[yy] - r_1[yy])

    area__ = abs(l_2[xx] - r_2[xx]) * abs(l_2[yy] - r_2[yy])

    x_dist = (min(r_1[xx], r_2[xx]) - max(l_1[xx], l_2[xx]))
 
    y_dist = (min(r_1[yy], r_2[yy]) - max(l_1[yy], l_2[yy]))
    
    area_ = 0
    
    if x_dist > 0 and y_dist > 0:
        area_ = x_dist * y_dist
    
    ar = (area_ + area__ - area_)
    return ar, area_

In [ ]:
def compute_iou(boxes1, boxes2):

    len_box1 = len(boxes1)
    len_box2 = len(boxes2)
    
    m = np.zeros((len_box1,len_box2))
    for i in range(len_box1):
        for j in range(len_box2):
            
            a,b = overlappingArea(np.array((boxes1[i][0],boxes1[i][1])),
                                  np.array((boxes1[i][2],boxes1[i][3])),np.array((boxes2[j][0],boxes2[j][1])),
                                  np.array((boxes2[j][2],boxes2[j][3])))
            m[i][j] = b/a
    
    return m

In [ ]:
array = compute_iou(b[124]['boxes'],b[125]['boxes'])
print(array)

In [ ]:
render_single_frame(a[124],b[124]['boxes'], colors=1)
render_single_frame(a[125],b[125]['boxes'], colors=1)

In [ ]:
for i in range(250):
    render_single_frame(a[i],b[i]['boxes'], colors=1)

In [ ]:
len(b[1]['boxes'])

### Q5.3 Matching with the Hungarian Algorithm (10 pts)

Given a matrix of similarities, the next step in tracking by detection is to find the bounding box that corresponds most to the previous frame. In essence, the idea is to find the bounding box that has the closest IoU to a bounding box in a previous frame. The challenging part is to remove this bounding box from contention for other matches. This problem is known as the optimal cost assignment problem. The Hungarian algorithm is the most commonly used method to solve this problem, and is implemented in ```scipy.optimize.linear_sum_assignment```. Below, you will implement a function ```compute_assignment``` that will produce such a matching. 

Some notes to keep in mind: ```scipy```'s implementation uses costs, so you will need to use the negative of the similarities you computed in the previous part. Additionally, since the IoU of a bounding box with itself is 1, you will need to set the diagonal entries of the cost matrix to some high value so they are not picked. Finally, it's likely there will be matches with very low IoU scores. Please use the ```threshold``` parameter to filter out any matches that are below ```threshold``` IoU.

Please submit your code for this section to the writeup PDF, along with the output run on the IOU matrix computed between the 124th and 125th images.

In [ ]:
def compute_assignment(iou, threshold=0.8):

    thresh = threshold
    shape0 = iou.shape[0]
    shape1 = iou.shape[1]
    b1 = np.zeros(shape0)
    b2 = np.zeros(shape1)
    
    for i in range(shape0):
        for j in range(shape1):
            if iou[i][j] > thresh:
                b1[i] = 1
                b2[j] = 1
    
    return np.array(b1), np.array(b2)

iou = compute_iou(b[124]["boxes"], b[125]["boxes"])
print("iou matrix", iou)
b1__iou,b2__iou = compute_assignment(iou, threshold = 0.8)
print("b1_iou" , b1__iou)
print("b2_iou" , b2__iou)

### Q5.4 Putting it all together (20 pts)

Now, you will put all the pieces together that you implemented above to create a full tracking system. You will maintain a set of tracks throughout the video. At the beginning, no tracks will exist, only detections. In each successive frame, you will read in the detections for that frame and associate the new detections to the previous ones, and create candidate tracks. If a candidate track has persisted for P frames, you will add it to the list of tracks. If a track has not had a match in K frames, you will remove it from the list of tracks. For each frame, you will render the bounding boxes corresponding to every track. Once a track is removed from the list, do not render its bounding box. 

The hyperparameters you will use for the tracker are:

P: number of frames a candidate track must exist before it is added to the list of tracks
K: number of frames a track must have no match for before it is removed from the list of tracks
iou_thresh: threshold to be used for whether a match is strong enough to be added to a track.


For your submission for this part, please submit 10 images (with bounding boxes) from successive frames at any point in the video. Bounding boxes belonging to the same track should be the same color. Please also submit your code to the writeup PDF. 

Please note that the output will NOT be perfect! There should be ID switches and the tracker, even if implemented correctly, will likely fail in several frames. This should hopefully demonstrate to you that tracking is a tough problem and show why it is a hot research area today :) 

In [ ]:
def run_tracker(images, boxes,rang, P, K, iou_thresh):

    all_frame0 = []
    all_frame1 = []
    all_frame2 = []
    a = images
    b = boxes
    
    for i in range(249):
        all_frame0.append(compute_iou(b[i]['boxes'], b[i+1]['boxes']))
    
    for i in range(249):
        b1, b2 = compute_assignment(all_frame0[i], iou_thresh)
        all_frame1.append(b1)
        all_frame2.append(b2)
        
    c = []
    for i in range(250-P):
        a_frame = all_frame1[i]
        u = a_frame.shape[0]
        array_u = np.zeros(u)
        
        for j in range(P):
            if all_frame1[i+j].shape[0] >= u:
                for k in range(u):
                    if a_frame[k] == 1 and all_frame1[i+j][k] == 1:
                        array_u[k] = array_u[k] + 1
            else:
                for o in range(all_frame1[i + j].shape[0]):
                    if a_frame[o] == 1 and all_frame1[i+j][o] == 1:
                        array_u[o] = array_u[o]+1
        c.append(array_u)
    import copy
    
    for i in range(len(c)):
        for j in range(c[i].shape[0]):
            if c[i][j] < P-K-1:
                c[i][j] = 0
    b_a = []
    b_d = []
    for i in range(len(c)):
        b_d = []
        for j in range(c[i].shape[0]):
            if c[i][j]>0:
                b_d.append(b[i]['boxes'][j])
        b_a.append(b_d)
    for i in range(len(c)):
        print("No of bounding boxes - ",len(b_a[i]))
        render_single_frame(a[i],b_a[i],rang)

p = run_tracker(a, b, c, 10, 3, 0.4)